## Setup and imports

In [1]:
!pip install -q easydict

In [ ]:
!git clone https://github.com/sayakpaul/Denoised-Smoothing-TF
!cd Denoised-Smoothing-TF && tar xf models.tar.gz

In [1]:
import sys
sys.path.append("Denoised-Smoothing-TF")

from scripts import certify, analyze
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

## Load models

In [2]:
print("Loading models.")
denoiser_mse = tf.keras.models.load_model("denoiser_mse")
denoiser_stab = tf.keras.models.load_model("denoiser_stability")
classifier = tf.keras.models.load_model("resnet20_classifier")
print("Models loaded.")

Loading models.
Models loaded.


## Load dataset

In [3]:
(x_train, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test.astype("float32") / 255.0
print(f"Total test examples: {len(x_test)}")

# Compute the mean and the variance of the training data for normalization
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(x_train/255.0) # Notice the scaling step

Total test examples: 10000


## Perform certification test with just the classifier

In [ ]:
# Classifier includes normalization layer
certify.perform_certification_test(classifier, dataset=(x_test, y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_no_denoiser")

The outputs have been removed for brevity. Here's how they should look like:

```
idx	label	predict	radius	correct	time
0	[3]	3	0.251	1	0:00:05.218241
20	[7]	-1	0.0	0	0:00:02.660233
40	[4]	1	0.158	0	0:00:02.642258
60	[7]	-1	0.0	0	0:00:02.648299
80	[8]	-1	0.0	0	0:00:02.683648
100	[4]	3	0.2	0	0:00:02.646552
```

## Prepend a denoiser with the classifier

Here, we are taking two denoisers trained with two different objectives: (i) MSE and (ii) Stability.

In [6]:
denoiser_mse_new = tf.keras.Model(denoiser_mse.input, denoiser_mse.output, name="denoiser_mse")
denoiser_mse_classifier = tf.keras.Sequential([
    denoiser_mse_new,
    classifier
], name="denoiser_mse_classifier")


denoiser_stab_new = tf.keras.Model(denoiser_stab.input, denoiser_stab.output, name="denoiser_stab")
denoiser_stab_classifier = tf.keras.Sequential([
    denoiser_stab_new,
    classifier
], name="denoiser_stab_classifier")

## Perform certification tests

In [ ]:
# Denoiser is appended which does not include a normalization layer
certify.perform_certification_test(denoiser_mse_classifier, dataset=(norm_layer(x_test), y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_denoiser_mse")

Here's how the output should look like:

```
idx	label	predict	radius	correct	time
0	[3]	3	0.292	1	0:00:06.420894
20	[7]	3	0.027	0	0:00:03.644317
40	[4]	-1	0.0	0	0:00:03.607284
60	[7]	4	0.135	0	0:00:03.600402
80	[8]	6	0.133	0	0:00:03.577247
100	[4]	6	0.175	0	0:00:03.588902
```

In [ ]:
# Denoiser is appended which does not include a normalization layer
certify.perform_certification_test(denoiser_stab_classifier, dataset=(norm_layer(x_test), y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_denoiser_stab")

Here's how the outputs should look like: 

```
idx	label	predict	radius	correct	time
0	[3]	3	0.53	1	0:00:04.103444
20	[7]	7	0.378	1	0:00:03.612182
40	[4]	0	0.377	0	0:00:03.602446
60	[7]	4	0.0825	0	0:00:03.594705
80	[8]	8	0.8	1	0:00:03.611531
100	[4]	-1	0.0	0	0:00:03.618659
```

## Generate the figures residing under `figures/`

In [9]:
certification_result_with_denoiser = "certification_output/sigma_0.25_denoiser_mse"
certification_result_without_denoiser = "certification_output/sigma_0.25_no_denoiser"

analyze.plot_certified_accuracy_per_sigma_best_model(
    "denoiser_mse", 'With vs Without Denoiser', 1.0,
    methods=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_with_denoiser), "$\sigma = 0.12$")],
    label='With Denoiser',
    methods_base=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_without_denoiser), "$\sigma = 0.12$")], 
    label_base='Without Denoiser',
    sigmas=[0.25])

In [10]:
certification_result_with_denoiser = "certification_output/sigma_0.25_denoiser_stab"

analyze.plot_certified_accuracy_per_sigma_best_model(
    "denoiser_stab", 'With vs Without Denoiser', 1.0,
    methods=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_with_denoiser), "$\sigma = 0.12$")],
    label='With Denoiser',
    methods_base=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_without_denoiser), "$\sigma = 0.12$")], 
    label_base='Without Denoiser',
    sigmas=[0.25])